In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load data
df = pd.read_csv('data\benin-malanville.csv', parse_dates=['Timestamp'])

ModuleNotFoundError: No module named 'pandas'

In [5]:
# Summary stats
print("Summary Statistics:")
display(df.describe())

# Missing values
missing = df.isna().sum()
print("\nMissing Values:")
display(missing[missing > 0])

# Columns with >5% nulls
print("\nColumns with >5% nulls:")
display(missing[missing > 0.05*len(df)])

Summary Statistics:


NameError: name 'df' is not defined

In [6]:
# Z-score calculation
numeric_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
z_scores = stats.zscore(df[numeric_cols].dropna())
outliers = (np.abs(z_scores) > 3).any(axis=1)

print(f"Found {outliers.sum()} outliers (|Z|>3)")

# Handle missing data
for col in numeric_cols:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

NameError: name 'stats' is not defined

In [7]:
# Daily trends
df.set_index('Timestamp', inplace=True)
fig, ax = plt.subplots(4, 1, figsize=(15, 12))
df['GHI'].resample('D').mean().plot(ax=ax[0], title='Daily GHI')
df['DNI'].resample('D').mean().plot(ax=ax[1], title='Daily DNI')
df['DHI'].resample('D').mean().plot(ax=ax[2], title='Daily DHI')
df['Tamb'].resample('D').mean().plot(ax=ax[3], title='Daily Temperature')
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

In [8]:
if 'Cleaning' in df.columns:
    clean_impact = df.groupby('Cleaning')[['ModA', 'ModB']].mean()
    clean_impact.plot(kind='bar', title='Module Performance Before/After Cleaning')
    plt.show()

NameError: name 'df' is not defined

In [9]:
# Heatmap
corr_matrix = df[['GHI', 'DNI', 'DHI', 'TModA', 'TModB']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Scatter plots
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
sns.scatterplot(data=df, x='WS', y='GHI', ax=ax[0])
sns.scatterplot(data=df, x='RH', y='Tamb', ax=ax[1])
sns.scatterplot(data=df, x='RH', y='GHI', ax=ax[2])
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

In [10]:
# Wind rose (requires windrose package: pip install windrose)
from windrose import WindroseAxes

ax = WindroseAxes.from_ax()
ax.bar(df['WD'].dropna(), df['WS'].dropna(), normed=True, opening=0.8, edgecolor='white')
ax.set_legend(title='Wind Speed (m/s)')
plt.show()


ModuleNotFoundError: No module named 'windrose'

In [11]:
# Bubble chart
plt.figure(figsize=(10, 6))
plt.scatter(df['Tamb'], df['GHI'], s=df['RH']*2, alpha=0.5)
plt.xlabel('Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.title('GHI vs Temperature (Bubble size = RH)')
plt.colorbar(label='Relative Humidity (%)')
plt.show()

NameError: name 'plt' is not defined

In [12]:
# Bubble chart
plt.figure(figsize=(10, 6))
plt.scatter(df['Tamb'], df['GHI'], s=df['RH']*2, alpha=0.5)
plt.xlabel('Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.title('GHI vs Temperature (Bubble size = RH)')
plt.colorbar(label='Relative Humidity (%)')
plt.show()

NameError: name 'plt' is not defined

In [13]:
# Ensure data directory exists
import os
os.makedirs('data', exist_ok=True)

# Export cleaned data
df.to_csv('data/benin_clean.csv')

NameError: name 'df' is not defined